# Test Algorithms 

1) Иерархическая кластеризация 
2) К — средних
3) DBSCAN
4) Разделения смеси гауссиан (EM). 


In [1]:
import numpy as np  
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow
import os
import random as rn 
import pandas as pd
import pymorphy2
import re
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer

SEED = 32 
os.environ['PYTHONHASHSEED']=str(SEED)
np.random.seed(SEED)
tensorflow.random.set_seed(SEED)
rn.seed(SEED)

### Load processed texts

In [2]:
#X_ = np.load('data_x_50.npy') 
#Y_ = np.load('data_y.npy')

In [3]:
#print(X_.shape)

In [4]:
#print(Y_.shape)

### Fetch 20 news groups

dataset

In [5]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
data_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=42) 
data_test = fetch_20newsgroups(subset='test',  shuffle=True, random_state=42)
print('data loaded')

data loaded


In [7]:
y_train, y_test = data_train.target, data_test.target

In [8]:
%%time
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5, stop_words='english')
X_train = vectorizer.fit_transform(data_train.data)
X_test = vectorizer.transform(data_test.data)

CPU times: user 3.93 s, sys: 9.32 ms, total: 3.94 s
Wall time: 3.94 s


In [9]:
from sklearn.decomposition import TruncatedSVD

def encode_svd(x, k=50): 
    svd_model = TruncatedSVD(n_components=k, algorithm='randomized', n_iter=100, random_state=42)
    x1 = svd_model.fit_transform(x) 
    return x1 

In [10]:
%%time
X_train_ = encode_svd(X_train)

CPU times: user 1min 48s, sys: 2min 36s, total: 4min 25s
Wall time: 1min 2s


In [11]:
y_train[y_train==1]
print(len(np.unique(y_train)), len(y_train))

20 11314


In [12]:
X_ = X_train_[:]
Y_ = y_train[:]

In [13]:

print(X_.shape)
print(Y_.shape)
print(np.unique(Y_))

(11314, 50)
(11314,)
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


Берем только 10 первых классов 0-9

In [14]:
yindex = Y_[Y_ < 10]
xindex = X_[Y_ < 10]
print(len(xindex), len(yindex))
X_ = xindex
Y_ = yindex

5790 5790


### Test

In [15]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import metrics 

def test_cluster(x,y, model):   
    model.fit(x)
    labels = model.labels_
    
    scores = []
    scores.append(metrics.homogeneity_score(y, labels))
    scores.append(metrics.completeness_score(y, labels))
    scores.append(metrics.v_measure_score(y, labels))
    scores.append(metrics.adjusted_rand_score(y, labels))
    scores.append(metrics.adjusted_mutual_info_score(y, labels,
                                               average_method='arithmetic'))
    try:
        scores.append(metrics.silhouette_score(x, labels, metric='sqeuclidean'))
    except ValueError:
        scores.append(0.0)
    scores.append(len(np.unique(labels)))
     
    print("Homogeneity: %0.3f" %  scores[0])
    print("Completeness: %0.3f" % scores[1])
    print("V-measure: %0.3f" % scores[2])
    print("Adjusted Rand Index: %0.3f"  % scores[3])
    print("Adjusted Mutual Information: %0.3f"  % scores[4])
    print("Silhouette Coefficient: %0.3f"  % scores[5])
    print("labels num: %0.3f"  % scores[6])
    return scores

### Подготавливаем массив для хранения результатов


In [16]:
results = []
n_clusters = len(np.unique(Y_))
print(n_clusters)

10


### Model KMeans

In [17]:

from sklearn.cluster import KMeans

m1 = KMeans(n_clusters=n_clusters, random_state=SEED)
r = test_cluster(X_, Y_, m1)
results.append(r)

Homogeneity: 0.430
Completeness: 0.519
V-measure: 0.470
Adjusted Rand Index: 0.239
Adjusted Mutual Information: 0.469
Silhouette Coefficient: 0.141
labels num: 10.000


### Agglomerative Clustering

In [18]:
from sklearn.cluster.hierarchical import AgglomerativeClustering
m2 = AgglomerativeClustering(n_clusters=n_clusters)
r = test_cluster(X_, Y_, m2)
results.append(r)

/home/vench/p37/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.cluster.hierarchical module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.cluster. Anything that cannot be imported from sklearn.cluster is now part of the private API.
  warnings.warn(message, FutureWarning)


Homogeneity: 0.292
Completeness: 0.444
V-measure: 0.353
Adjusted Rand Index: 0.118
Adjusted Mutual Information: 0.350
Silhouette Coefficient: 0.118
labels num: 10.000


### DBSCAN

In [19]:
from sklearn.cluster import DBSCAN
m3 = DBSCAN(eps=0.212, min_samples = 2)
r = test_cluster(X_, Y_, m3)
results.append(r)

Homogeneity: 0.012
Completeness: 0.266
V-measure: 0.023
Adjusted Rand Index: 0.001
Adjusted Mutual Information: 0.018
Silhouette Coefficient: 0.637
labels num: 8.000


### GaussianMixture

In [20]:
from sklearn.mixture import GaussianMixture

class GM:
    
    def __init__(self, model):
        self.model = model
        self.labels_ = []
    
    def fit(self, x):
        self.model.fit(x)
        self.labels_ = self.model.predict(x)
        
        

m4 = GaussianMixture(n_components=n_clusters)
r = test_cluster(X_, Y_,GM(m4))
results.append(r)

Homogeneity: 0.362
Completeness: 0.402
V-measure: 0.381
Adjusted Rand Index: 0.243
Adjusted Mutual Information: 0.379
Silhouette Coefficient: -0.003
labels num: 10.000


### Экспериментальный алгоритм

In [25]:

from sklearn.metrics.pairwise import euclidean_distances
import numpy as np 

class Cluster:
    
    def __init__(self, n):
        ''' '''
        self.n = n # номер класетра (начальной точки)
        self.nodes = set([n]) # объединенные с кластером точки 
        self.join_n  = -1 # номер кластера с которым слит
        self.dist = -1 # расстояние при слиянии (dist <= 0)

    def active(self):
        ''' '''
        return self.join_n == -1
        
    def merge(self, c, dist):
        ''' '''
        self.nodes = self.nodes.union(c.nodes) 
        c.join_n = self.n
        c.dist = np.abs(dist)
        
    def get_n(self):
        ''' '''
        if self.join_n == -1:
            return self.n
        return self.join_n
        
class HierarchicalClustering:
    
    def __init__(self, alpha = 1.01, max_iteration = 200, debug= False, delta=0., stop_neg_sum = True, betta = 2.,n_clusters=1):
        ''' '''
        self.alpha = alpha
        self.betta = betta
        self.max_iteration = max_iteration
        self.debug = debug
        self._c_all = []
        self.delta = delta
        self.stop_neg_sum = stop_neg_sum
        self.n_clusters= n_clusters
        self.labels_ = []
            
    
    def fit(self, x, min_delta = 1.e-7):
        ''' '''
        self._c_all = []
        y_ = []
        M =  euclidean_distances(x,x)
        C = []
        size = len(M)
        for i in range(size): 
            C.append( Cluster(i) )
        
        delta = self.delta 
        
        for i in range(self.max_iteration): 
            d = np.min(M[M > min_delta])  * self.alpha
            #d = self._get_min(M, C) * self.alpha
            if d > delta:
                delta = d  
            #delta = delta     
            if(self.debug): 
                print('delta: %.8f, d: %.8f' % (delta, d))    
            M_ = M - delta
            ''' join clusters '''
            join = False
        
            for i in range(size):  
                for j in range(i+1,size):  
                    
                    if(M[i][j] <= 0 or M_[i][j] > 0):
                        continue 
                        
                    if C[i].dist > np.abs(M_[i][j]): 
                        continue
                        
                    a = C[i].get_n()
                    b = C[j].get_n() 
                    if(a == b):
                        continue 
            
                    C[a].merge(C[b], M_[i][j]) 
                    for s in C[b].nodes:
                        #C[s].join_n = a
                        C[a].merge(C[s], M_[i][j])
                    join = True        
            if join == False: 
                delta = delta * self.betta 
                continue
            
            M =   M_ 
                    
            if len(M[M > min_delta]) == 0:
                if(self.debug): 
                    print('len(M[M > min_delta]) == 0') 
                break
               
            y_ = np.zeros(size)
            cl = 0
            for c in C:
                if(c.active() == True): 
                    for i in c.nodes:
                        y_[i] = cl
                    cl = cl + 1
                    
            self._c_all.append(y_)  
          
            if len(np.unique(y_)) <= self.n_clusters: 
                if(self.debug): 
                    print('len(np.unique(y_)) <= self.n_clusters') 
                break
            # func    
            neg,pos = [],[]    
            for i in range(size): 
                for j in range(i,size):
                    #if C[i].active() == False:
                    #    continue
                    #if C[j].active() == False:
                    #    continue
                        
                    if(M[i][j] <= 0):
                        neg.append(np.abs(M[i][j]) + delta)
                    elif(M[i][j] > 0):
                        pos.append(M[i][j])    
            if(self.debug):
                print('Sum pos: %.3f, sum neg: %.3f,Sum pos2: %.3f, sum neg2: %.3f, Std pos: %.3f, Std neg: %.3f, n_cls: %d' % 
                      (sum(pos) , sum(neg),sum(pos)/len(pos) , sum(neg)/len(neg),np.std(pos), np.std(neg), len(np.unique(y_)))) 
            if np.std(pos) == 0:
                break
            if self.stop_neg_sum and sum(pos) < sum(neg): 
                print('sum(pos) < sum(neg)')
                break 
        self.labels_ = y_         
        return y_                
        
    def print_name(self):
        print('Hierarchical clustering')
        
    
# ---
class HC:
    
    def __init__(self, alpha = 1.01, max_iteration = 200, debug= False, delta=0., stop_neg_sum = True, betta = 2.,n_clusters=1):
        ''' '''
        self.alpha = alpha
        self.betta = betta
        self.max_iteration = max_iteration
        self.debug = debug
        self._c_all = []
        self.delta = delta
        self.stop_neg_sum = stop_neg_sum
        self.n_clusters= n_clusters
        self.labels_ = []
    
    def log(self, msg):
        if(self.debug):
            print(msg)
            
    def print_name(self):
        self.log('HC')
    
    def fit(self, x): 
        ''' '''
        A = euclidean_distances(x,x)
        #print(A)
        self._c_all = []
        C = A.copy()
        labels = [n +1 for n in range(0,len(x))]
        #print(labels)
        n = 1 
        mim_1 = 0
        for i in range(self.max_iteration):
            if len(A[A>0]) == 0:
                break
            mim_ = min(A[A>0]) * self.alpha
            if mim_ > mim_1:
                mim_1 = mim_
            mim_1   = mim_   
            
            #A = A - mim_1  
            A[(A - mim_1 <= 0) & (C > 0)] = -n
            C = C - mim_1
            w1,w2 = np.where(A == -n) 
            #print(A)
            lab= len(np.unique(labels))
            for j in range(0, len(w1)):
                #print((w1[j],w2[j])) !!!
                labels[w1[j]] = labels[w2[j]]
                #A[w2[j],:]=0
                #A[:,w1[j]]=0
            #print(labels)
            if len(np.unique(labels)) == lab:
                continue
            self._c_all.append(labels.copy())
            if(len(np.unique(labels)) <= self.n_clusters):
                break
            self.log('min: %.10f, len: %d' % (mim_1, len(np.unique(labels))))     
            #if sum(C[C > 0]) < sum(C[C < 0] * -1):
            #    self.log("By D")
            #    print("By D")
            #    break
            n = n + 1
        #print(labels)
        #print(C)
        self.labels_ = labels
        return labels




In [26]:
m5 = HC(alpha=4.7,n_clusters=1,debug=True, stop_neg_sum=True)
r = test_cluster(X_, Y_, m5)
results.append(r) 

min: 0.0000000088, len: 5788
min: 0.0109430680, len: 5765
min: 0.0525544133, len: 5629
min: 0.2473783173, len: 250
Homogeneity: 0.232
Completeness: 0.210
V-measure: 0.220
Adjusted Rand Index: 0.013
Adjusted Mutual Information: 0.165
Silhouette Coefficient: -0.111
labels num: 250.000


In [ ]:
A = np.array([[0,2,3], [0,0,4], [0,0,0]])
x = np.array([[1,1], [2,2,], [3,3],[1,4]])
A = X_ #euclidean_distances(x,x)
#print(A)
C = A.copy()
n = 1
alpha = 1.05
mim_1 = 0
while(True):
    if len(A[A>0]) == 0:
        break
    mim_ = min(A[A>0]) * alpha
    if mim_ > mim_1:
        mim_1 = mim_
  
    print('%.10f %d' % (mim_, len(A[A>0])))
    A = A - mim_1              
    A[A <= 0] = 0 
    C[(A == 0) & (C > 0)] = -n
    n = n + 1

print(C)


In [ ]:
from sklearn.decomposition import PCA, SparsePCA, NMF

p = PCA(n_components=2)
x1 = p.fit_transform(X_)


plt.rcParams["figure.figsize"] = (20,10)
plt.scatter(x1[:,0], x1[:,1], c=m5._c_all[-8])

In [ ]:
r

In [ ]:
df = pd.read_pickle('data/dftime_cat.pkl')

In [ ]:
r = np.array(m5._c_all[-4])
u = np.unique(r)

for i in u:
    indexs = np.where(r == i)[0]
    print('Cluster: %d, len: %d' % (i, len(indexs)))
    for n in indexs[0:3]:
        t,d = df.iloc[n]['title'],  df.iloc[n]['text2']
        print('N: %d, title: %s' % (n, t))
    print()    
        
        #print(df['title'][n])
 

### Таблица результатов

In [ ]:
df2 = pd.DataFrame(results, columns=[ 
    'Homogeneity', 
    'Completeness', 
    'V-measure', 
    'Adjusted Rand Index', 'Adjusted Mutual Information', 'Silhouette Coefficient', 'len'])
df2.head(len(results))

In [ ]:
from scipy.stats import ttest_ind
#from scipy.stats import 

?scipy.stats.t.ppf

In [ ]:
np.round(df2.values, 2)

### Тест 2

Сравним 4 алгоритма на синтетических наборах данных

- HierarchicalClustering и DBSCAN как адаптивные алгоритмы

- HierarchicalClustering и AgglomerativeClustering как аглоритмически близкие

In [ ]:
from sklearn  import datasets
 
#dx, dy    
def test_2algo(func_ds, test_alg):
    result1,result2 = [],[] 

    for n in range(100): 
        bx,by = func_ds()
        n_clusters = len(np.unique(by))
        print(n_clusters)
        
        m5 = HierarchicalClustering(alpha=1,n_clusters=n_clusters,debug=0)
        r = test_cluster(bx, by, m5)
        result1.append(r) 

        m3 = test_alg #DBSCAN(min_samples = 2) #DBSCAN(eps=0.103, min_samples = 2)
        r = test_cluster(bx, by, m3)
        result2.append(r)

    df2_1 = pd.DataFrame(result1, columns=[ 
    'Homogeneity', 
    'Completeness', 
    'V-measure', 
    'Adjusted Rand Index', 'Adjusted Mutual Information', 'Silhouette Coefficient', 'len'])
    df2_2 = pd.DataFrame(result2, columns=[ 
    'Homogeneity', 
    'Completeness', 
    'V-measure', 
    'Adjusted Rand Index', 'Adjusted Mutual Information', 'Silhouette Coefficient', 'len'])
    
    ###
    # 200 - 2 = 198 => 180-199	1.973 # http://medstatistic.ru/theory/t_cryteria.html
    # http://medstatistic.ru/theory/t_cryteria.html
    ss = 1.973
    df2_1.replace([np.inf, -np.inf], np.nan)
    df2_2.replace([np.inf, -np.inf], np.nan)
    df2_1.fillna(0)
    df2_2.fillna(0)
    for c in df2_1.columns:
        
        # print(df2_1[c].values)
        # print(df2_2[c].values)
        tStat = ttest_ind(df2_1[c].values, df2_2[c].values)
        z = "<"
        if df2_1[c].mean() > df2_2[c].mean():
            z = ">"
        print('%s: important: %s, alg1: %.4f, alg2: %.4f %s %.4f' % (c, np.abs(tStat.statistic) > ss, tStat.statistic, df2_1[c].mean(), z, df2_2[c].mean()))


In [ ]:
 

?datasets.make_circles

####  2 класса

- make_blobs
- moons

### DBSCAN VS test

In [ ]:
 
def ds2():
    return datasets.make_blobs(n_samples=100,  n_features=3, cluster_std=1 + np.random.rand())
 
def ds2moon():
    return datasets.make_moons(n_samples=100,   noise=.05)

# dep
def ds2b2():
    return datasets.make_blobs(n_samples=100,cluster_std=[1.0, 2.5, 0.5],random_state=60)

def ds2circl():
    return datasets.make_circles(n_samples=100, factor=.5, noise=.05, random_state=np.random.randint(1,255))

def ds2len(): 
    X, y = datasets.make_blobs(n_samples=100, random_state=np.random.randint(1,255))
    transformation = [[0.6, -0.6], [-0.4, 0.8]]
    x = np.dot(X, transformation)
    return x,y


test_2algo(ds2, DBSCAN(eps=0.212, min_samples = 2))

In [ ]:
test_2algo(ds2moon, DBSCAN(eps=0.212, min_samples = 2)) 

In [ ]:
#test_2algo(ds2b2, DBSCAN(eps=0.212, min_samples = 2)) 

In [ ]:
test_2algo(ds2circl, DBSCAN(eps=0.212, min_samples = 2)) 

In [ ]:
test_2algo(ds2len, DBSCAN(eps=0.212, min_samples = 2))  

### Test vs AgglomerativeClustering

In [ ]:
_,yy = ds2()
np.unique(yy)

In [ ]:
test_2algo(ds2, AgglomerativeClustering(n_clusters = 3))

In [ ]:
_,yy = ds2moon()
np.unique(yy)

In [ ]:
test_2algo(ds2moon, AgglomerativeClustering(n_clusters = 2))

In [ ]:
_,yy = ds2b2()
np.unique(yy)

In [ ]:
#test_2algo(ds2b2, AgglomerativeClustering(n_clusters = 3))

In [ ]:
_,yy = ds2circl()
np.unique(yy)

In [ ]:
test_2algo(ds2circl, AgglomerativeClustering(n_clusters = 2))

In [ ]:
_,yy = ds2len()
np.unique(yy)

In [ ]:
test_2algo(ds2circl, AgglomerativeClustering(n_clusters = 3))

#### 10 класса


In [ ]:
def ds10():
    return datasets.make_classification(n_classes=10, n_informative=10)

test_2algo(ds10, DBSCAN(eps=0.212, min_samples = 2))

In [ ]:
test_2algo(ds10, AgglomerativeClustering(n_clusters = 10))

In [ ]:
#print(result3)